# Annotator/Evaluator Development Template

Use this template to iterate on a new annotator. Adjust the settings as needed.

## Settings and imports

In [ ]:
from modelplane.runways import responder, annotator

In [ ]:
sut_id = "llama-3.1-8b-instruct-turbo-together"
experiment = "experiment_name"
prompts = "airr_official_1.0_demo_en_us_prompt_set_release_reduced.csv"
cache_dir = None
n_jobs = 4

## Define a new annotator

In [ ]:
import random

from pydantic import BaseModel

from modelgauge.annotator import CompletionAnnotator
from modelgauge.annotator_registry import ANNOTATORS
from modelgauge.annotators.llama_guard_annotator import LlamaGuardAnnotation
from modelgauge.single_turn_prompt_response import TestItem
from modelgauge.sut import SUTResponse
from modelgauge.tests.safe_v1 import BaseSafeTestVersion1

class AnnotationRequest(BaseModel):
    test_item: TestItem
    response: SUTResponse


class NewAnnotator(CompletionAnnotator[LlamaGuardAnnotation]):
    """Simple random implementation for demonstration. Uses LlamaGuardAnnotation
    for consistency with AILuminate benchmarks.

    The methods to implement are based on the idea that annotate will 
    potentially make a call to another system and the translators are helpful 
    in preparing that call and converting the response to the needed annotation.
    """
    # Probability of unsafe per category
    PROB_UNSAFE = 0.01

    def translate_request(self, test_item: TestItem, response: SUTResponse):
        return AnnotationRequest(test_item=test_item, response=response)

    def annotate(self, annotation_request: AnnotationRequest) -> LlamaGuardAnnotation:
        unsafe_categories = []
        for category in BaseSafeTestVersion1.hazards:
            if random.random() < self.PROB_UNSAFE:
                unsafe_categories.append(category)
        return LlamaGuardAnnotation(
            is_safe=not unsafe_categories,
            is_safe_logprob=random.random(),
            violation_categories=unsafe_categories,
        )

    def translate_response(self, request: AnnotationRequest, response: LlamaGuardAnnotation) -> LlamaGuardAnnotation:
        return response


annotator_id = "new_annotator"
ANNOTATORS.register(NewAnnotator, annotator_id)

## Run the model

Save this run_id to avoid having to re-run the model later. The results are saved as an artifact in mlflow.

NOTE: you may need to replace http://mlflow:8080 with http://localhost:8080 in the generated mlflow links.

In [ ]:
run_id = responder.respond(
    sut_id=sut_id,
    experiment=experiment,
    prompts=prompts,
    cache_dir=cache_dir,
    n_jobs=n_jobs,
)

## Annotate the model

The evaluation will be available in mlflow, and the artifact will be saved with that mlflow run for inspection of the output jsonl.

NOTE: you may need to replace http://mlflow:8080 with http://localhost:8080 in the generated mlflow links.

In [ ]:
annotator.annotate(
    annotator_id=annotator_id,
    experiment=experiment,
    response_run_id=run_id,
    cache_dir=cache_dir,
    n_jobs=n_jobs,
)